# 1. Objetivo
O objetivo deste notebook é reproduzir a metodologia do cálculo de consumo aparente, apresentado no artigo [Indicadores de Consumo Aparente de Bens Industriais: metodologia e resultados](http://www.ipeadata.gov.br/doc/td_2101.pdf).

**Autor:** Amilton Lobo Mendes Júnior
amilton.mendes@gmail.com

# 2. Preparação do ambiente


## 2.1 Instalação de bibliotecas utilizadas

In [75]:
!pip install pyIpeaData

## 2.2. Import das bibliotecas utilizadas neste notebook





In [1]:
import pandas as pd
import requests
import json
from pandas import json_normalize
from datetime import datetime
import pyIpeaData as ip

## 2.3 URL das tabelas de recursos e usos de 2021, armazenada no github.

In [2]:
url_recursos = 'https://github.com/amiltonmendes/consumo-aparente/raw/refs/heads/main/MIP/68_tab1_2019.xls'
url_usos = 'https://github.com/amiltonmendes/consumo-aparente/raw/refs/heads/main/MIP/68_tab2_2019.xls'


## 2.4 Dicionários com informações sobre os dados por setor

In [3]:
#Criar um mapa que faça a correlação entre um código de cnae e códigos para baixar a série de importação, exportação e a url para download de dados do sidra

mapa_cnae = {
    "2100": {  # CNAE 2100 - Fabricação de produtos farmacêuticos
        "importacao": "FUNCEX12_MQFARM2N12",
        "exportacao": "FUNCEX12_XQFARM2N12",
        "sidra_url": "https://apisidra.ibge.gov.br/values/t/8888/n1/all/v/12606/p/all/c544/129330/d/v12606%205"
    },
    "29" : {
        "importacao": "FUNCEX12_MQVEIC2N12",
        "exportacao": "FUNCEX12_XQVEIC2N12",
        "sidra_url": "https://apisidra.ibge.gov.br/values/t/8888/n1/all/v/12606/p/all/c544/129338/d/v12606%205"
    }
    # adicione mais CNAEs aqui...
}



1. Inicialmente, foram carregados os dados de produção e importação da tabela de recursos (Tabela 1 das TRU's).
2. Em seguida, tanto para importação quanto para produção foram somados todos os z produtos pertencentes a um mesmo setor, conforme fórmula abaixo:

$$
    \sum_{n=1}^{z}a_n
$$
3. Com isso, obtemos os valores totais de produção e importação por setor a preços básicos, restando as informações de exportação, que são carregados da tabela de usos (Tabela 2 das TRU's).

4. A exportação está disponibilizada a preços do consumidor, ou seja, com o acréscimo de margem de transporte, margem de comércio e impostos, que devem ser subtraídos a fim de que tenhamos uma informação comparável com os valores de importação e produção, ou seja, a preços básicos. Para isso, se faz necessário calcular o percentual que o preço básico representa do preço do consumidor. Essa razão é o fator de ajuste, que é multiplicado ao valor de exportação.

5. De posse dos valores de exportação, importação e produção das contas nacionais, obtêm-se a participação de cada uma dessas variáveis de acordo com as fórmulas:

$$
    w_p = \frac{P}{P-X+I}
    \\ 

    \\

    w_x = \frac{X}{P-X+I}

    \\

    w_i = \frac{I}{P-X+I}

$$



**Onde:**
$$
\begin{aligned}

I : \text{Valor de importação do setor}
\\
P : \text{Valor de produção do setor}
\\
X : \text{Valor de exportação do setor}
\end{aligned}
$$

## 2.5 Funções e classes utilizadas no notebook

In [45]:
#Carrega dados oferta
def carrega_dados_oferta():
  oferta = pd.read_excel(url_recursos, sheet_name='oferta',skiprows=3,skipfooter=2,dtype={'Unnamed: 0' : str})
  #Renomear as duas primeiras colunas que não foram carregadas por estarem mescladas
  oferta= oferta.rename(columns={oferta.columns[0] : 'Código do produto', oferta.columns[1] : 'Descrição do produto'})
  #Remover a primeira linha (vazia)
  oferta = oferta.iloc[1:]
  return oferta
def carrega_dados_producao():
  producao = pd.read_excel(url_recursos, sheet_name='producao',skiprows=3,skipfooter=2,dtype={'Unnamed: 0' : str})
  producao = producao.rename(columns={producao.columns[0] : 'Código do produto', producao.columns[1] : 'Descrição do produto'})
  producao = producao.iloc[1:]
  return producao

def carrega_dados_importacao():
  importacao = pd.read_excel(url_recursos, sheet_name='importacao',skiprows=3,skipfooter=5,dtype={'Unnamed: 0' : str})
  importacao = importacao.rename(columns={importacao.columns[0] : 'Código do produto', importacao.columns[1] : 'Descrição do produto'})
  importacao = importacao.iloc[1:]
  return importacao

def carrega_dados_exportacao():
  exportacao = pd.read_excel(url_usos, sheet_name='demanda',skiprows=3,skipfooter=2,dtype={'Unnamed: 0' : str})
  exportacao = exportacao.rename(columns={exportacao.columns[0] : 'Código do produto', exportacao.columns[1] : 'Descrição do produto'})
  exportacao = exportacao.iloc[1:]
  return exportacao

class ConsumoAparente():
  def __init__(self,oferta,producao,importacao,exportacao):
    self.oferta = oferta
    self.producao = producao
    self.importacao = importacao
    self.exportacao = exportacao
  def carrega_dados_producao_mensal(self,url):
    r = requests.get(url)
    #Apresentar data de carregamento
    a = json.loads(r.text)
    res = json_normalize(a)

    producao = pd.DataFrame(res)
    nomes_colunas = producao.iloc[0].values
    producao.columns = nomes_colunas
    producao = producao.iloc[1:]
    producao['Mês (Código)'] = producao['Mês (Código)'].astype(str)
    producao['Ano'] = producao['Mês (Código)'].str[:4]
    producao['Mes'] = producao['Mês (Código)'].str[4:]

    producao['Ano'] = producao['Ano'].astype(int)
    producao['Mes'] = producao['Mes'].astype(int)

    producao = producao[['Ano','Mes','Valor']]
    producao = producao.rename(columns={'Valor' : 'producao'})

    producao['producao'] = producao['producao'].astype(float)

    return producao.copy()
  def carrega_dados_quantum_exp_imp(self,url,movimento):
    ##Carrega dados de quantum de exportação e importação
    dados_movimento = ip.get_serie(url)

    dados_movimento['VALDATA'] = pd.to_datetime(dados_movimento['VALDATA'],utc=True)

    dados_movimento['Ano'] = dados_movimento['VALDATA'].dt.year
    dados_movimento['Mes'] = dados_movimento['VALDATA'].dt.month

    dados_movimento = dados_movimento[['Ano','Mes','VALVALOR']]

    dados_movimento = dados_movimento.rename(columns={'VALVALOR' : movimento})
    dados_movimento[movimento] = dados_movimento[movimento].astype(float)

    return dados_movimento

  def calcula_consumo_aparente(self,cnae,verbose=0,ano_base = 2012):
    preco_consumidor,preco_basico = self.oferta[self.oferta['Código do produto'].str.startswith(cnae)][['Oferta total\na preço\nde consumidor','Oferta total\n a preço básico ']].sum().values
    fator_ajuste = preco_basico/preco_consumidor
    valor_producao = self.producao[self.producao['Código do produto'].str.startswith(cnae)][['Total\ndo produto']].sum().values[0]
    valor_exportacao = self.exportacao[self.exportacao['Código do produto'].str.startswith(cnae)][['Exportação\nde bens e\nserviços (1)']].sum().values[0]
    valor_exportacao = valor_exportacao*fator_ajuste
    valor_importacao = self.importacao[self.importacao['Código do produto'].str.startswith(cnae)][['Importação de bens\ne serviços (1)']].sum().values[0]
    #Calcular pesos de acordo com fórmula constante do arquivo
    total = valor_importacao - valor_exportacao + valor_producao
    peso_importacao = valor_importacao/total
    peso_exportacao = valor_exportacao/total
    peso_producao = valor_producao/total

    # Normalizar os pesos para que a soma seja 100
    soma_pesos = peso_importacao + peso_exportacao + peso_producao
    peso_importacao = (peso_importacao / soma_pesos) * 100
    peso_exportacao = (peso_exportacao / soma_pesos) * 100
    peso_producao = (peso_producao / soma_pesos) * 100    

    # Normalizar os pesos para que a soma seja 1 e arredondar para 2 casas decimais
    soma_pesos = peso_importacao + peso_exportacao + peso_producao
    peso_importacao = round((peso_importacao / soma_pesos), 2)
    peso_exportacao = round((peso_exportacao / soma_pesos), 2)
    peso_producao = round((peso_producao / soma_pesos), 2)

    # Ajustar o último peso para garantir que a soma seja exatamente 1
    ajuste = 1 - (peso_importacao + peso_exportacao + peso_producao)
    if ajuste != 0:
        peso_producao += ajuste

    #Prepara dados de quantum de exportação e importação para a junção com dados de produção e cálculo de consumo aparente
    if verbose == 1:
      print(f'Preço de consumidor: {preco_consumidor}')
      print(f'Preço básico: {preco_basico}')
      print(f'Fator de ajuste: {fator_ajuste}')
      print(f'Peso da importação: {peso_importacao}')
      print(f'Peso da exportação: {peso_exportacao}')
      print(f'Peso da produção: {peso_producao}')
      print('Carregando dados de produção da PIM-PF (Sidra)')
    producao = self.carrega_dados_producao_mensal(mapa_cnae[cnae]['sidra_url'])
    #Converter esse índice para a base de selecionada
    media_indice_base_selecionada = producao[producao['Ano'] == ano_base]['producao'].mean()
    producao['producao'] = producao['producao']*100/media_indice_base_selecionada

    if verbose == 1:
      print('Carregando dados de exportação e importação da Quantum ( Funcex - Ipeadata)')
    dados_exportacao = self.carrega_dados_quantum_exp_imp(mapa_cnae[cnae]['exportacao'],'exportacao')
    #Converter esse índice para a base de selecionada
    media_indice_base_selecionada = dados_exportacao[dados_exportacao['Ano'] == ano_base]['exportacao'].mean()
    dados_exportacao['exportacao'] = dados_exportacao['exportacao']*100/media_indice_base_selecionada
    dados_importacao = self.carrega_dados_quantum_exp_imp(mapa_cnae[cnae]['importacao'],'importacao')
    #Converter esse índice para a base de selecionada
    media_indice_base_selecionada = dados_importacao[dados_importacao['Ano'] == ano_base]['importacao'].mean()
    dados_importacao['importacao'] = dados_importacao['importacao']*100/media_indice_base_selecionada

    #Juntar os dados de produção, importação e exportação
    producao = producao.merge(dados_exportacao,how='inner',on=['Ano','Mes']).merge(dados_importacao,how='inner',on=['Ano','Mes'])



    producao['producao'] = producao['producao'].astype(float)
    producao['importacao'] = producao['importacao'].astype(float)
    producao['exportacao'] = producao['exportacao'].astype(float)

    producao['consumo_aparente'] = producao['producao']*peso_producao - producao['exportacao']*peso_exportacao + producao['importacao']*peso_importacao

    producao['consumo_aparente_12_meses_anterior'] = producao['consumo_aparente'].shift(12)
    producao['variacao_consumo_aparente'] = ((producao['consumo_aparente']/producao['consumo_aparente_12_meses_anterior'])-1)*100

    producao['consumo_aparente_12m'] = producao['consumo_aparente'].rolling(window=12).sum()
    producao['producao_12m'] = producao['producao'].rolling(window=12).sum()
    producao['importacao_12m'] = producao['importacao'].rolling(window=12).sum()

    # Calcular a variação percentual em relação aos 12 meses anteriores
    producao['variacao_consumo_aparente_12m'] = (
        (producao['consumo_aparente_12m'] / producao['consumo_aparente_12m'].shift(12)) - 1
    ) * 100
    producao['variacao_producao_12m'] = (
        (producao['producao_12m'] / producao['producao_12m'].shift(12)) - 1
    ) * 100
    producao['variacao_importacao_12m'] = (
        (producao['importacao_12m'] / producao['importacao_12m'].shift(12)) - 1
    ) * 100

    # Arredondar os valores para 2 casas decimais
    producao['variacao_consumo_aparente_12m'] = producao['variacao_consumo_aparente_12m'].round(2)
    producao['variacao_producao_12m'] = producao['variacao_producao_12m'].round(2)
    producao['variacao_importacao_12m'] = producao['variacao_importacao_12m'].round(2)


    #Calcular a variação da produção e importação acumulada dos 12 meses anteriores
    


    return producao






In [46]:
consumoAparente = ConsumoAparente(carrega_dados_oferta(),carrega_dados_producao(),carrega_dados_importacao(),carrega_dados_exportacao())

In [47]:
consumoAparente.calcula_consumo_aparente('29',0,ano_base=2021).tail(13)[['consumo_aparente','variacao_consumo_aparente','variacao_consumo_aparente_12m','variacao_producao_12m','variacao_importacao_12m']]

,consumo_aparente,variacao_consumo_aparente,variacao_consumo_aparente_12m,variacao_producao_12m,variacao_importacao_12m
265,81.524671,10.868147,-3.40,-6.13,5.06
266,90.088503,-0.200384,-3.44,-6.66,5.48
267,101.636080,37.185664,-0.30,-3.90,7.68
268,93.385690,3.569473,-0.26,-4.70,9.41
269,108.411484,34.362234,2.79,-3.70,18.97
270,101.882934,31.939077,5.98,-0.83,21.00
271,108.057077,16.425363,8.25,1.33,22.84
272,102.096517,18.499054,11.19,4.47,24.65
273,114.153131,32.378826,14.36,7.66,26.93
274,98.637536,15.184462,16.80,10.55,27.46


In [38]:
consumoAparente.calcula_consumo_aparente('2100',0,ano_base=2019).tail(13)[['consumo_aparente','variacao_consumo_aparente','consumo_aparente_12_meses_anterior']]

,consumo_aparente,variacao_consumo_aparente,consumo_aparente_12_meses_anterior
265,96.464852,-12.570714,110.334713
266,104.300955,-23.447959,136.248431
267,108.510679,-12.722958,124.329006
268,123.332733,6.922561,115.347717
269,114.492271,7.920777,106.089184
270,122.482596,14.753406,106.735477
271,121.850753,-0.630247,122.623585
272,110.187628,4.169732,105.777010
273,122.012365,3.551960,117.827190
274,107.944177,4.260677,103.532971


In [28]:
producao = consumoAparente.carrega_dados_producao_mensal(mapa_cnae['2100']['sidra_url'])


In [31]:
producao.tail(14)

,Ano,Mes,producao
265,2024,1,72.81993
266,2024,2,82.38201
267,2024,3,92.89157
268,2024,4,100.29527
269,2024,5,103.71688
270,2024,6,107.13063
271,2024,7,102.94700
272,2024,8,110.04800
273,2024,9,102.27364
274,2024,10,109.16742


In [32]:
variacao_producao_12m = producao['producao'].pct_change(periods=12)*100
producao['variacao_producao_12m'] = variacao_producao_12m
producao['variacao_producao_12m'] = producao['variacao_producao_12m'].round(2)
producao['variacao_producao_12m'] = producao['variacao_producao_12m'].astype(float)
producao['variacao_producao_12m'] = producao['variacao_producao_12m'].astype(str).str.replace('nan','-')
producao['variacao_producao_12m'] = producao['variacao_producao_12m'].str.replace('inf','-')
producao['variacao_producao_12m'] = producao['variacao_producao_12m'].str.replace('-','-')



In [34]:
producao.tail(14)

,Ano,Mes,producao,variacao_producao_12m
265,2024,1,72.81993,-17.24
266,2024,2,82.38201,-18.73
267,2024,3,92.89157,-16.41
268,2024,4,100.29527,-6.55
269,2024,5,103.71688,1.98
270,2024,6,107.13063,16.26
271,2024,7,102.94700,6.06
272,2024,8,110.04800,-6.72
273,2024,9,102.27364,3.37
274,2024,10,109.16742,4.68
